In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 55.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
model = AutoModelForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=5)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")  # Or return a default loss

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-10 07:38:29,436] A new study created in memory with name: no-name-9d8c1063-1621-406f-bab7-1b41c9d146db
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.509654,0.779841,0.000000,0.000000,0.000000
1,No log,0.470567,0.806366,0.205572,0.156627,0.177792
2,No log,0.445256,0.815385,0.476094,0.216867,0.257434
3,No log,0.419654,0.835013,0.434912,0.327711,0.373325
4,No log,0.404955,0.838196,0.419268,0.371084,0.393019
5,0.468800,0.397834,0.837666,0.588064,0.385542,0.402676
6,0.468800,0.395141,0.839788,0.589475,0.387952,0.409892


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-10 07:40:57,057] Trial 0 finished with value: 0.4098916584014872 and parameters: {'learning_rate': 5.602116361979629e-06, 'batch_size': 8, 'num_train_epochs': 7, 'weight_decay': 0.009469152601464225}. Best is trial 0 with value: 0.4098916584014872.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featur

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.382880,0.845093,0.744649,0.404819,0.435549
1,No log,0.377759,0.846684,0.708111,0.433735,0.472739
2,No log,0.369658,0.853581,0.695691,0.453012,0.509671
3,No log,0.367204,0.854111,0.687025,0.467470,0.520444
4,No log,0.365153,0.853050,0.682006,0.479518,0.529849
5,No log,0.365073,0.854111,0.677565,0.496386,0.542798
6,No log,0.361431,0.854642,0.668682,0.496386,0.545403
7,No log,0.361239,0.858355,0.679244,0.513253,0.562105
8,No log,0.360471,0.856233,0.666540,0.510843,0.557312


...

[I 2024-12-10 08:00:24,085] Trial 8 finished with value: 0.7239758782189217 and parameters: {'learning_rate': 3.7383656955332994e-06, 'batch_size': 16, 'num_train_epochs': 8, 'weight_decay': 0.027886297197748402}. Best is trial 7 with value: 0.7307927186893883.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: 

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.561619,0.868435,0.713878,0.703614,0.702749
1,No log,0.545330,0.874271,0.726894,0.689157,0.703164
2,No log,0.544247,0.879045,0.720402,0.718072,0.715705
3,No log,0.565892,0.879045,0.727482,0.703614,0.705391
4,No log,0.550653,0.883289,0.742279,0.715663,0.722329


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


[I 2024-12-10 08:01:55,670] Trial 9 finished with value: 0.7223291009643972 and parameters: {'learning_rate': 4.1185076833617694e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.005580440087521341}. Best is trial 7 with value: 0.7307927186893883.


Best trial:
FrozenTrial(number=7, state=TrialState.COMPLETE, values=[0.7307927186893883], datetime_start=datetime.datetime(2024, 12, 10, 7, 56, 18, 412916), datetime_complete=datetime.datetime(2024, 12, 10, 7, 57, 58, 327098), params={'learning_rate': 2.9310939011754772e-05, 'batch_size': 4, 'num_train_epochs': 4, 'weight_decay': 0.023123646580599278}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=7, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 752


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.604899,0.874801,0.747620,0.684337,0.710076
1,No log,0.595153,0.883820,0.749541,0.720482,0.726803
2,0.033200,0.539122,0.894960,0.766915,0.749398,0.755607
3,0.033200,0.546852,0.896021,0.773039,0.742169,0.754900


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=752, training_loss=0.025669975166625166, metrics={'train_runtime': 98.8904, 'train_samples_per_second': 60.956, 'train_steps_per_second': 7.604, 'total_flos': 792643404172800.0, 'train_loss': 0.025669975166625166, 'epoch': 4.0})

In [13]:
eval_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 377
  Batch size = 8


### Model Saving

In [14]:
!mkdir 'best_model_codebert-base_python'
!mkdir 'best_model_codebert-base_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_codebert-base_python')
tokenizer.save_pretrained('./best_model_codebert-base_python_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_codebert-base_python')
tokenizer = AutoTokenizer.from_pretrained('./best_model_codebert-base_python_tokenizer')

Saving model checkpoint to ./best_model_codebert-base_python
Configuration saved in ./best_model_codebert-base_python/config.json
Model weights saved in ./best_model_codebert-base_python/pytorch_model.bin
tokenizer config file saved in ./best_model_codebert-base_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_codebert-base_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_codebert-base_python/config.json
Model config RobertaConfig {
  "_name_or_path": "./best_model_codebert-base_python",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id":

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 1, 0, 0, 0]) array([0, 0, 1, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([1, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 406
  Batch size = 8


Evaluation Metrics: {'accuracy': 0.8763546798029557, 'precision': 0.7258897832321698, 'recall': 0.6995412844036697, 'f1': 0.7101437250920383}
Inference Time: 1.87 seconds


In [18]:
!zip -r best_model_codebert-base_python.zip './best_model_codebert-base_python'
!zip -r best_model_codebert-base_python_tokenizer.zip './best_model_codebert-base_python_tokenizer'

  adding: best_model_codebert-base_python/ (stored 0%)
  adding: best_model_codebert-base_python/training_args.bin (deflated 51%)
  adding: best_model_codebert-base_python/pytorch_model.bin (deflated 7%)
  adding: best_model_codebert-base_python/config.json (deflated 53%)
  adding: best_model_codebert-base_python_tokenizer/ (stored 0%)
  adding: best_model_codebert-base_python_tokenizer/tokenizer_config.json (deflated 73%)
  adding: best_model_codebert-base_python_tokenizer/vocab.json (deflated 59%)
  adding: best_model_codebert-base_python_tokenizer/special_tokens_map.json (deflated 51%)
  adding: best_model_codebert-base_python_tokenizer/merges.txt (deflated 53%)
  adding: best_model_codebert-base_python_tokenizer/tokenizer.json (deflated 82%)


In [20]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_codebert-base_python.zip "/content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python/"
!cp best_model_codebert-base_python_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_folder = '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python/best_model_codebert-base_python.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python_Tokenizer/best_model_codebert-base_python_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python/best_model_codebert-base_python.zip
replace ./best_model_codebert-base_python/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_codebert-base_python/training_args.bin  
  inflating: ./best_model_codebert-base_python/pytorch_model.bin  
  inflating: ./best_model_codebert-base_python/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/codebert-base_Python_Tokenizer/best_model_codebert-base_python_tokenizer.zip
replace ./best_model_codebert-base_python_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_codebert-base_python_tokenizer/tokenizer_config.json  
  inflating: ./best_model_codebert-base_python_tokenizer/vocab.json  
  inflating: ./best_model_codebert-base_python_tokenizer/special_tokens_map.json  
  inflating: ./best_model_codebert-base_python_tokenizer/merges.txt  
  inflating: ./best_model_codebert-base_python_

In [23]:
best_model_codebert_base_python = 'best_model_codebert-base_python'
best_model_codebert_base_python_tokenizer = 'best_model_codebert-base_python_tokenizer'

In [27]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_codebert_base_python)
tokenizer = AutoTokenizer.from_pretrained(best_model_codebert_base_python_tokenizer)

loading configuration file best_model_codebert-base_python/config.json
Model config RobertaConfig {
  "_name_or_path": "best_model_codebert-base_python",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab

In [28]:
text = "for detecting automatically generated fields."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

SequenceClassifierOutput(loss=None, logits=tensor([[-8.9365,  2.2405, -3.1644, -9.0560, -4.9929]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 2
